# Main Idea of applying Quantum GNNs

## We could encode graph information (Nodes and Edges) into quantum states and then manipulate these states using quantum gates to perform computations. The QGNN can then output a quantum state, which can be measured to produce a prediction. For example , Each node in the graph is represented as a feature vector, and the edges between the nodes are represented by a series of entangling CNOT gates. For training,we can apply rotation gates at the nodes (which are qubits) and train these values using an optimizer. And for output,we can map these values to a classical output answer using a threshold of 0.5

In [2]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 97.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 86.1 MB/s eta 0:00:00


### Obtaining Edge lists from task 2 (not used for main task)

In [10]:
!pip install energyflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.5/700.5 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.0/503.0 KB 45.0 MB/s eta 0:00:00


In [11]:
import energyflow
dataset = energyflow.qg_jets.load(num_data=100000, pad=True, ncol=4, generator='pythia',with_bc=False, cache_dir='~/.energyflow')                    
X = dataset[:-1][0]
y = dataset[-1]
sum_array_1 = []
sum_array_2 = []
sum_array_3 = []
sum_array_4 = []
for i in range(X.shape[0]):
    temp_1 = 0
    temp_2 = 0
    temp_3 = 0
    temp_4 = 0
    for j in range(X.shape[1]):
        if(X[i][j][0]!=0):
            temp_1 += X[i][j][0]
        if(X[i][j][1]!=0):
            temp_2 += X[i][j][1]
        if(X[i][j][2]!=0):
            temp_3 += X[i][j][2]
        if(X[i][j][3]!=0):
            temp_4 += X[i][j][3]
    sum_array_1.append(temp_1)
    sum_array_2.append(temp_2)
    sum_array_3.append(temp_3)
    sum_array_4.append(temp_4)
mean_1 = sum(sum_array_1)/len(sum_array_1)
mean_2 = sum(sum_array_2)/len(sum_array_2)
mean_3 = sum(sum_array_3)/len(sum_array_3)
mean_4 = sum(sum_array_4)/len(sum_array_4)
bool_1 = [i<=mean_1 for i in sum_array_1]
bool_2 = [i<=mean_2 for i in sum_array_2]
bool_3 = [i<=mean_3 for i in sum_array_3]
bool_4 = [i<=mean_4 for i in sum_array_4]
binary_1 = []
for i in bool_1:
    if(i==True):
        binary_1.append(1)
    else:
        binary_1.append(0)
binary_2 = []
for i in bool_2:
    if(i==True):
        binary_2.append(1)
    else:
        binary_2.append(0)
binary_3 = []
for i in bool_3:
    if(i==True):
        binary_3.append(1)
    else:
        binary_3.append(0)
binary_4 = []
for i in bool_4:
    if(i==True):
        binary_4.append(1)
    else:
        binary_4.append(0)
binary = []
for i in range(len(binary_1)):
    binary.append(binary_1[i] + binary_2[i] + binary_3[i] + binary_4[i])
connections_0 = []
connections_1 = []
connections_2 = []
connections_3 = []
connections_4 = []
for i in range(len(binary)):
    if(binary[i]==0):
        connections_0.append(i)
    if(binary[i]==1):
        connections_1.append(i)
    if(binary[i]==2):
        connections_2.append(i)
    if(binary[i]==3):
        connections_3.append(i)
    if(binary[i]==4):
        connections_4.append(i)
labels = []
labels.append('Quarks')
labels.append('Gluons')
nodes = []
N = X.shape[0]
F = X.shape[2]
print('X shape: ', X.shape)
edge_list=[]
for i in range(len(connections_1)-1):
    edge_list.append((connections_1[i],connections_1[i+1]))
for i in range(len(connections_2)-1):
    edge_list.append((connections_2[i],connections_2[i+1]))
for i in range(len(connections_3)-1):
    edge_list.append((connections_3[i],connections_3[i+1]))
for i in range(len(connections_4)-1):
    edge_list.append((connections_4[i],connections_4[i+1]))

X shape:  (100000, 139, 4)


In [14]:
len(edge_list)

94572

## Implementing QGNN circuit on dummy data (Main task)

In [9]:
import pennylane as qml
from pennylane import numpy as np

dev = qml.device("default.qubit", wires=X.shape[0]) #simulating qubits on a classical computer

@qml.qnode(dev) #defining quantum node
def qgnn_circuit(edge_list, features):
    for edge in edge_list:
        qml.CNOT(wires=[edge[0], edge[1]])

    for i in range(len(features)):
        qml.RY(features[i], wires=i)

    for edge in edge_list:
        qml.CNOT(wires=[edge[0], edge[1]])

    return [qml.expval(qml.PauliZ(i)) for i in range(len(features))]

# Define the edge list and features for a simple graph
edge_list = [(0, 1), (1, 2), (2, 3), (3, 0)]
features = np.array([0.5, 0.2, -0.1, 0.7])

# Execute the QGNN circuit on the edge list and features
result = qgnn_circuit(edge_list, features)

# Map the expectation values to a binary classification output
threshold = 0.0
output = [1 if expval > threshold else 0 for expval in result]

print("Expectation values:", result)
print("Binary classification output:", output)

Expectation values: [0.74585141 0.86008934 0.85579247 0.65454619]
Binary classification output: [1, 1, 1, 1]


In [8]:
print(qml.draw(qgnn_circuit)(edge_list, features))

0: ─╭●───────╭X──RY(0.50)──╭●───────╭X─┤  <Z>
1: ─╰X─╭●────│───RY(0.20)──╰X─╭●────│──┤  <Z>
2: ────╰X─╭●─│───RY(-0.10)────╰X─╭●─│──┤  <Z>
3: ───────╰X─╰●──RY(0.70)────────╰X─╰●─┤  <Z>


# Exploring code for Using quark-gluon dataset to train a QGNN model

### Although not part of the main task,I was curious to see how far i can go with implementing a QGNN model on the quark gluon dataset

In [15]:
import pennylane as qml
from pennylane import numpy as np
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the QGNN circuit
dev = qml.device("default.qubit", wires=X.shape[1])
@qml.qnode(dev)
def qgnn_circuit(features):
    for edge in edge_list:
        qml.CNOT(wires=[edge[0], edge[1]])

    for i in range(len(features)):
        qml.RY(features[i], wires=i)

    for edge in edge_list:
        qml.CNOT(wires=[edge[0], edge[1]])

    return [qml.expval(qml.PauliZ(i)) for i in range(len(features))]

# Define the QGNN classifier
def qgnn_classifier(features):
    probs = qgnn_circuit(features)
    return probs[0]

# Define the QGNN loss function
def qgnn_loss(features, label):
    prediction = qgnn_classifier(features)
    return (label - prediction) ** 2

# Initialize the QGNN weights
weights = np.random.randn(X.shape[1])

# Train the QGNN using gradient descent
lr = 0.1
epochs = 1000
for epoch in range(epochs):
    for i in range(X_train.shape[0]):
        features = X_train[i]
        label = y_train[i]
        grad_fn = qml.grad(qgnn_loss, argnum=0)
        weights = weights - lr * grad_fn(features, label)

    if epoch % 100 == 0:
        loss = 0
        for i in range(X_test.shape[0]):
            features = X_test[i]
            label = y_test[i]
            loss += qgnn_loss(features, label)
        print(f"Epoch {epoch}: loss = {loss/X_test.shape[0]}")

# Test the QGNN on the test set
correct = 0
total = X_test.shape[0]
for i in range(total):
    features = X_test[i]
    label = y_test[i]
    prediction = qgnn_classifier(features)
    if prediction >= 0.5 and label == 1:
        correct += 1
    elif prediction < 0.5 and label == 0:
        correct += 1

accuracy = correct / total
print(f"Test accuracy: {accuracy}")

ValueError: ignored

### Unfortunately,the number of qubits are extremely large to finish the entire implementation on this large dataset. Once these qubits are obtained, these features would be quantum encoded for a single data point. The circuit consists of several layers of quantum gates, each of which acts on pairs of qubits that are connected by an edge in the graph. In this case, we are using a simple two-qubit gate, the CNOT gate, for each edge in the graph.We then measure the expectation value of the Pauli-Z operator on each qubit in the circuit, which measures the spin of the qubit along the z-axis,which is trained on the gradient descent optimizer of pennylane. This gives us a real number that represents the output of the QGNN circuit for the given input of features. The expectation values can be mapped onto a classical output by thresholding them at a certain value(which is 0.5 in our case).